In [24]:
import pandas as pd 
import numpy as np

In [25]:
df=pd.read_csv('Animal_csv.csv')
df.head()

,id,text,label
0,0,We started giving this to my 5 year old Labrad...,dogs
1,1,This product is easy to set up and use. I have...,dogs
2,2,my dog has nerves and wants to itch and chew.....,dogs
3,3,the breeded gave us a can of Nupro when we too...,dogs
4,4,"Very study, well made poop bag. Easy to open a...",dogs


### The  data is text, so not much EDA can be achieved
- It is not exactly a balanced  data set as 60% is dogs and 40 % is cats

In [26]:
df['label'].value_counts()/len(df)

label
dogs    0.602168
cats    0.397832
Name: count, dtype: float64

##### Encoding the label numerically

In [27]:
df['label']=df['label'].apply(lambda x: 1 if x=="dogs" else 0)

In [28]:
x=df['text']
y=df['label']
y.value_counts()

label
1    9333
0    6166
Name: count, dtype: int64

### Train test split: into train , validation,test
-80:20:20

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)
x_train,x_val,y_train,y_val= train_test_split(x_train, y_train, test_size=0.2, random_state=42,stratify=y_train)


### Create bag-of-words NLP technique
- We also know that there is a class imbalance
- Therefore, we will be using  SMOTE
- However a lot of people implement it through a pipeline


In [30]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score,accuracy_scor

In [31]:
result=pd.DataFrame(columns=['Name','F1-train','F1-Val','F1-Test','Acc-train','Acc-Val','Acc-Test'])
def append_score(df, name, f1_train, f1_val, f1_test, acc_train, acc_val, acc_test):
    new_row_data = {
        'Name': name,
        'F1-train': f1_train,
        'F1-Val': f1_val,
        'F1-Test': f1_test,
        'Acc-train': acc_train,
        'Acc-Val': acc_val,
        'Acc-Test': acc_test
    }

    # Use .loc to assign the new row at the next available integer index
    # len(df) gives you the current number of rows, which is the next available integer index.
    df.loc[len(df)] = new_row_data

    return df # Return the modified DataFrame
    

In [38]:
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Step 1: Vectorize text
    ('smote', SMOTE(random_state=42)),  # Step 2: Apply SMOTE
    ('classifier', MultinomialNB()) # Step 3: Train your classifier (e.g., Naive Bayes)
])
pipeline2=Pipeline([
    ('tfidf', TfidfVectorizer()),  # Step 1: Vectorize text
    ('smote', SMOTE(random_state=42)),  # Step 2: Apply SMOTE
    ('classifier', BernoulliNB()) # Step 3: Train your classifier (e.g., Naive Bayes)
])
pipes=[]
pipes.append(pipeline1)
pipes.append(pipeline2)
names=["Multinomial","Bernoulli"]

In [49]:
def function(result,pipes,names,x_train,x_val,x_test,y_train,y_val,y_test):
    for i in range(len(pipes)):
        pipes[i].fit(x_train,y_train)
        train_pred=pipes[i].predict(x_train)
        test_pred=pipes[i].predict(x_test)
        val_pred=pipes[i].predict(x_val)
        f1_train=f1_score(y_train,train_pred)
        f1_val=f1_score(y_val,val_pred)
        f1_test=f1_score(y_test,test_pred)
        acc_train=accuracy_score(y_train,train_pred)
        acc_val=accuracy_score(y_val,val_pred)
        acc_test=accuracy_score(y_test,test_pred)
        result=append_score(result,names[i],f1_train,f1_val,f1_test,acc_train,acc_val,acc_test)
        
    return result
        
        
        

In [39]:
result=function(result,pipes,names,x_train,x_val,x_test,y_train,y_val,y_test)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE(random_state=42)),
                ('classifier', MultinomialNB())])

In [40]:
result

### Using Bermnoulli

In [46]:
pipeline2.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE(random_state=42)),
                ('classifier', BernoulliNB())])

,Name,F1-train,F1-Val,F1-Test,Acc-train,Acc-Val,Acc-Test
0,Multi,0.903148,0.834507,0.840441,0.887085,0.810484,0.817742
1,Bernoulli,0.784457,0.769321,0.773746,0.702490,0.682258,0.684194


### Conclusion:
-  Scores on multonimial model is definitely higher than Bernoulli
- But multinomial model tends to overfit more